In [1]:
#Import libraries
from ibmpairs import paw
from datetime import datetime, timedelta
import re, os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import datetime
import zipfile
from zipfile import ZipFile
import glob
import time

In [2]:
#Add credentials
PAIRS_SERVER      = 'https://pairs.res.ibm.com'
PAIRS_USER        = 'guzmnque@ualberta.ca'
PAIRS_CREDENTIALS = (
    PAIRS_USER, paw.get_pairs_api_password(
        PAIRS_SERVER, PAIRS_USER, passFile='credentials.txt'
    )
)

In [4]:
#ERA5 layers to evaluate
#49451 Soil water (7 to 21 cm) (m3 m-3)
#49427 Low cloud cover
#49423 Temperature (K) (2 meter temperature)
#49430 Maximum temperature (K)
#49429 Minimum temperature
#49422 Dewpoint (K) (2 meter dewpoint temperature)
#49439 Surface pressure (Pa)
#49316 name='Precip-CHIRPS' ; unit='mm'
#49447 Thermal radiation J m-2 (Surface thermal radiation downwards)
#49462 Thermal radiation (top of atmosphere) (Outgoing Longwave Radiation)
#49440 Solar radiation (J m-2) (Surface solar radiation downwards)
#49455 Rain water content of atmosphere (kg m-2)

In [5]:
#Import TMCF coordinates
df = pd.read_csv('GIS/TMCF.csv')
df

,long,lat,id,tmcf,country,realm
0,-155.589264,-14.966681,0,Ulla Ulla,BOL,Neotropic
1,-105.816653,-17.750014,1,Amboró,BOL,Neotropic
2,-105.000014,-17.266653,2,Carrasco,BOL,Neotropic
3,-104.166653,-20.000014,3,Tariquía,BOL,Neotropic
4,-103.616681,-22.383319,4,Itatiaia,BRA,Neotropic
...,...,...,...,...,...,...
524,149.600014,2.199681,524,Hose Mountains,MYS,Indomalayan
525,150.283042,2.322625,525,Linau Balui Plateau,MYS,Indomalayan
526,150.549597,2.896486,526,Penambo Range,MYS,Indomalayan
527,152.057653,3.306375,527,Dulit Range,MYS,Indomalayan


In [8]:
#Create a coord list for coords which have values
nround=6

#round(df.iloc[0]['latitude'],nround)

coords=[]
for i in range(len(df)):
    coords.append(str(round(df.iloc[i]['lat'],nround)))
    coords.append(str(round(df.iloc[i]['long'],nround)))

In [9]:
# Helper function to calculate the slope for each pixel
# dset = dlayer id
# agg = aggregration
# startDate = starting date
# duration = aggregation duration in days
# N = numbers of years
# aoi - needs a number but not used here

def generateQueryJson(dset,agg,startDate,duration,N,coords):
#calculate the average annual value
    queryJson = {
        'layers' : [
            {
                'alias' : 'Y{0}'.format(str(years+startDate.year).zfill(2)),
                'aggregation' : agg,
                'type' : 'raster', 
                'id' : dset,
                'temporal' : {'intervals' : [{
                    'start' : (startDate.replace(startDate.year + years)).strftime(iso8601),
                    'end' : (startDate.replace(startDate.year + years) + timedelta(days = duration)).strftime(iso8601)
                }]},
                'output' : False
            }
            for years in range(0,N) ],
        "spatial" : {"type" : "point","coordinates" : coords},
        'temporal' : {'intervals' : [{'snapshot' : startDate.strftime(iso8601)}]},
        "batch" : True
    }

#calculate the mean of x     
    queryJson['layers'].extend([{
        'alias' : 'xm',
        'expression' : '('+'+'.join([
            '{0}'.format(str(years+startDate.year).zfill(2))
            for years in range(0,N)
        ] 
        )}
    ])
    queryJson['layers'][N]['expression']=queryJson['layers'][N]['expression']+')'+'/'+str(N)
    queryJson['layers'][N].update({'output' : False})
    
#calculate the mean of y     
    queryJson['layers'].extend([{
        'alias' : 'ym',
        'expression' : '('+'+'.join([
            '$Y{0}'.format(str(years+startDate.year).zfill(2))
            for years in range(0,N)
        ] 
        )}
    ])
    queryJson['layers'][N+1]['expression']=queryJson['layers'][N+1]['expression']+')'+'/'+str(N)
    queryJson['layers'][N+1].update({'output' : False})
    
#calculate Sxx     
    queryJson['layers'].extend([{
        'alias' : 'Sxx',
        'expression' : ' + '.join([
            '({0}-$xm)*({0}-$xm)'.format(str(years+startDate.year).zfill(2))
            for years in range(0,N)
        ] 
        )}
    ])    
    queryJson['layers'][N+2].update({'output' : False})

#calculate Syy     
    queryJson['layers'].extend([{
        'alias' : 'Syy',
        'expression' : ' + '.join([
            '($Y{0}-$ym)*($Y{0}-$ym)'.format(str(years+startDate.year).zfill(2))
            for years in range(0,N)
        ] 
        )}
    ])
    queryJson['layers'][N+3].update({'output' : False})
        
#calculate Sxy     
    queryJson['layers'].extend([{
        'alias' : 'Sxy',
        'expression' : ' + '.join([
            '({0}-$xm)*($Y{0}-$ym)'.format(str(years+startDate.year).zfill(2))
            for years in range(0,N)
        ] 
        )}
    ])
    queryJson['layers'][N+4].update({'output' : False})
    
#calculate slope
    queryJson['layers'].extend([{
        'alias' : 'slope',
        'expression' : '$Sxy/$Sxx'
        }]
   )  

#calculate standard error 
    queryJson['layers'].extend([{
        'alias' : 'error',
        'expression' : 'math:sqrt(($Syy-($Sxy*$Sxy/$Sxx))/($Sxx*'+str(N-2)+'))'
        }]
   )  
    return queryJson

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

In [10]:
#Define periods of time and agregation time
startDate = datetime.date(1997,1,1)
days = 365
N = 24
aggregation = 'Mean'

In [17]:
#Select dataset product and generate query
dset = 49455
queryJson = generateQueryJson(dset,aggregation,startDate,days,N,coords)
queryJson

{'layers': [{'alias': 'Y1997',
   'aggregation': 'Mean',
   'type': 'raster',
   'id': 49455,
   'temporal': {'intervals': [{'start': '1997-01-01T00:00:00Z',
      'end': '1998-01-01T00:00:00Z'}]},
   'output': False},
  {'alias': 'Y1998',
   'aggregation': 'Mean',
   'type': 'raster',
   'id': 49455,
   'temporal': {'intervals': [{'start': '1998-01-01T00:00:00Z',
      'end': '1999-01-01T00:00:00Z'}]},
   'output': False},
  {'alias': 'Y1999',
   'aggregation': 'Mean',
   'type': 'raster',
   'id': 49455,
   'temporal': {'intervals': [{'start': '1999-01-01T00:00:00Z',
      'end': '2000-01-01T00:00:00Z'}]},
   'output': False},
  {'alias': 'Y2000',
   'aggregation': 'Mean',
   'type': 'raster',
   'id': 49455,
   'temporal': {'intervals': [{'start': '2000-01-01T00:00:00Z',
      'end': '2000-12-31T00:00:00Z'}]},
   'output': False},
  {'alias': 'Y2001',
   'aggregation': 'Mean',
   'type': 'raster',
   'id': 49455,
   'temporal': {'intervals': [{'start': '2001-01-01T00:00:00Z',
      

In [18]:
#Download query from PAIRS
query = paw.PAIRSQuery(queryJson, PAIRS_SERVER, PAIRS_CREDENTIALS)
query.submit()
query.poll_till_finished()
query.download()